In [1]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

data = {}

with open('tetris_train.json') as json_file:
    data['train'] = json.load(json_file)

with open('tetris_test.json') as json_file:
    data['test'] = json.load(json_file)


labels = {
    'S': [0,0,0,0,1],
    'T': [0,0,0,1,0],
    'L': [0,0,1,0,0],
    'O': [0,1,0,0,0],
    'I': [1,0,0,0,0]
}

X_train = []
Y_train = []
X_test = []
Y_test = []


for key in data['train'].keys():
    X_train.extend(data['train'][key])
    for _ in range(len(data['train'][key])):
        Y_train.append(labels[key])

for key in data['test'].keys():
    X_test.extend(data['test'][key])
    for _ in range(len(data['test'][key])):
        Y_test.append(labels[key])


X_train = torch.FloatTensor(X_train)
Y_train = torch.FloatTensor(Y_train)
X_test = torch.FloatTensor(X_test)
Y_test = torch.FloatTensor(Y_test)

print(X_train.size())
print(Y_train.size())
print(X_test.size())
print(Y_test.size())

# print(X_train[-5:], Y_train[-5:])

torch.Size([800, 3, 3])
torch.Size([800, 5])
torch.Size([200, 3, 3])
torch.Size([200, 5])


In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
        self.mp1 = nn.MaxPool2d(3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(2, 3, 3, padding=1)
        self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
        self.mp = nn.MaxPool2d(2, stride=1)
        self.fc = nn.Linear(27, 5)

    def forward(self, x):
        in_size = x.size(0)
        x = self.mp1(F.relu(self.conv1(x)))
        x = self.mp2(F.relu(self.conv2(x)))       
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.1)

#
# Print number of parameters
#
# for item in list(ConvNet().parameters()):
#     print (item)
# print(list(ConvNet().parameters()))
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(ConvNet())

217

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(10000):  # loop over the dataset multiple times

    running_loss = 0.0
    dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(dataset=dataset,
                          batch_size=256,
                          shuffle=True,
                          num_workers=1)
    
    for _, data in enumerate(train_loader, 0):
#     for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = X_train[i], torch.tensor(Y_train[i])
        inputs, labels = data
#         inputs = inputs.view(256, 1, 3, 3)
        for i in range(len(labels)):
            x = inputs[i].view(1, 1, 3, 3)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, labels[i])
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
    
    if epoch % 100 == 99:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

#         correct = 0.
#         total = float(len(Y_test))

#         with torch.no_grad():
#             for i in range(len(Y_test)):
#                 images, labels = X_test[i], Y_test[i]
#                 images = images.view(1, 1, 3, 3)

#                 output = model(images)
#                 predicted = output.data
#                 correct += [int(item == predicted[0].max()) for item in predicted[0]] == labels.int().tolist()
#                 accuracy = 100. * correct / total
#         print('Accuracy of the network: %.2f %%' % (accuracy))

#         results[epoch+1] = [running_loss, accuracy]

Epoch: 100, Loss: 141.853
Epoch: 200, Loss: 138.864
Epoch: 300, Loss: 141.783
